# Fetch Camera Metadata

Sadly, the metadata associated the camera and lens that took each photo isn't included in the dataset I downloaded from Macaulay. However, I can scrape the data from the Macaulay website. Since it's only ~3,000 photos that I'm interested in, I shouldn't raise any red flags.

In [1]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime

In [2]:
top_records_df = pd.read_csv("../results/top_photographs.csv")
top_records_df.head()

,catalog_number,common_name,scientific_name,photographer,date,year,rating,num_ratings,location,normalized_rating
0,418857311,Marbled Duck,Marmaronetta angustirostris,David Fernández,2022-02-20,2022,5.0,25,Europe,5.0
1,144333121,Crescent-chested Warbler,Oreothlypis superciliosa,Patrick Van Thull,2019-03-08,2019,5.0,31,North America,5.0
2,622728122,Coppersmith Barbet,Psilopogon haemacephalus,John Russell,2024-08-20,2024,5.0,62,Asia,5.0
3,512323911,Yellow-rumped Cacique,Cacicus cela,Charlie Bostwick,2022-01-06,2022,5.0,27,South America,5.0
4,205901211,Yellow-throated Bunting,Emberiza elegans,Sicheng Wan,2018-01-26,2018,5.0,22,Asia,5.0


In [3]:
def scrape_macaulay_technical_info(catalog_numbers, base_url="https://macaulaylibrary.org/asset/{}", delay=1.0):
    """
    Scrape technical information from Macaulay Library pages.
    
    Parameters
    ----------
    catalog_numbers : list
        List of catalog numbers to scrape (e.g., ["123456", "789012"])
    base_url : str, optional
        Template URL with {} placeholder for catalog number
        Default: "https://macaulaylibrary.org/asset/{}"
    delay : float, optional
        Time in seconds to wait between requests
        Default: 1.0
    
    Returns
    -------
    pandas.DataFrame
        DataFrame containing technical information
    """
    all_data = []
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Research Bot (Responsible scraping for academic research)'
    })
    
    for catalog_number in catalog_numbers:
        try:
            # Make request
            url = base_url.format(catalog_number)
            response = session.get(url)
            response.raise_for_status()
            
            # Parse HTML
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Find the "Technical information" heading
            tech_heading = soup.find('h3', string='Technical information')
            if not tech_heading:
                print(f"No technical information heading found for {catalog_number}")
                continue
                
            # Get the definition list that follows the heading
            tech_section = tech_heading.find_next('dl', class_='defList')
            
            if tech_section:
                # Extract information
                tech_info = {'catalog_number': catalog_number}
                terms = tech_section.find_all('dt')
                definitions = tech_section.find_all('dd')
                
                for term, definition in zip(terms, definitions):
                    column_name = term.text.strip().lower().replace(' ', '_')
                    tech_info[column_name] = definition.text.strip()
                
                all_data.append(tech_info)
            
            # Respect the site by waiting between requests
            time.sleep(delay)
            
        except Exception as e:
            print(f"Error processing {catalog_number}: {str(e)}")
            continue
    
    return pd.DataFrame(all_data)

def scrape_in_batches(catalog_numbers, batch_size=500, delay=1.0, checkpoint_dir="checkpoints"):
    """
    Scrape data in batches with checkpointing.
    
    Parameters
    ----------
    catalog_numbers : list
        List of all catalog numbers to process
    batch_size : int, optional
        Number of items to process per batch (default: 500)
    delay : float, optional
        Delay between requests in seconds (default: 1.0)
    checkpoint_dir : str, optional
        Directory to save checkpoint files (default: "checkpoints")
    
    Returns
    -------
    pandas.DataFrame
        Combined results from all batches
    """
    # Create checkpoint directory if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    # Timestamp for this run
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    results_list = []
    total_batches = len(catalog_numbers) // batch_size + (1 if len(catalog_numbers) % batch_size else 0)
    
    for i in range(0, len(catalog_numbers), batch_size):
        batch_num = i // batch_size + 1
        batch = catalog_numbers[i:i + batch_size]
        
        print(f"\nStarting batch {batch_num} of {total_batches}")
        print(f"Processing catalog numbers {i} to {i + len(batch)}")
        
        # Scrape batch
        batch_results = scrape_macaulay_technical_info(batch, delay=delay)
        results_list.append(batch_results)
        
        # Save checkpoint
        checkpoint_path = os.path.join(
            checkpoint_dir, 
            f'macaulay_results_{timestamp}_batch_{batch_num}_of_{total_batches}.csv'
        )
        batch_results.to_csv(checkpoint_path, index=False)
        print(f"Saved checkpoint: {checkpoint_path}")
        
        # Save combined results so far
        combined_path = os.path.join(
            checkpoint_dir,
            f'macaulay_results_{timestamp}_combined.csv'
        )
        pd.concat(results_list).to_csv(combined_path, index=False)
        print(f"Updated combined results: {combined_path}")
        
        if batch_num < total_batches:
            print(f"Taking a 10 second break before next batch...")
            time.sleep(10)
    
    return pd.concat(results_list)

In [4]:
catalog_numbers = top_records_df['catalog_number'].tolist()
print("Starting scraping process...")
print(f"Total catalog numbers: {len(catalog_numbers)}")
print(f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Run the scraper with batching
results = scrape_in_batches(
    catalog_numbers,
    batch_size=250,
    delay=.25,
    checkpoint_dir="../results/checkpoints"
)

print("\nScraping completed!")

Starting scraping process...
Total catalog numbers: 3292
Current time: 2024-12-31 19:29:54

Starting batch 1 of 14
Processing catalog numbers 0 to 250
Saved checkpoint: ../results/checkpoints/macaulay_results_20241231_192954_batch_1_of_14.csv
Updated combined results: ../results/checkpoints/macaulay_results_20241231_192954_combined.csv
Taking a 10 second break before next batch...

Starting batch 2 of 14
Processing catalog numbers 250 to 500
Saved checkpoint: ../results/checkpoints/macaulay_results_20241231_192954_batch_2_of_14.csv
Updated combined results: ../results/checkpoints/macaulay_results_20241231_192954_combined.csv
Taking a 10 second break before next batch...

Starting batch 3 of 14
Processing catalog numbers 500 to 750
Saved checkpoint: ../results/checkpoints/macaulay_results_20241231_192954_batch_3_of_14.csv
Updated combined results: ../results/checkpoints/macaulay_results_20241231_192954_combined.csv
Taking a 10 second break before next batch...

Starting batch 4 of 14
Pr

This takes roughly 1.5 hours to run. I could make it run faster, but I want to be careful not to spam the Macaulay website.

Now, lets do a final bit of processing and save the results. We don't need to keep rows that are missing the camera information. We also want to join the results back to the original data.

In [5]:
filtered_results = results[results['model'].notnull()]
print(f"There are {len(results)-len(filtered_results)} records with missing camera model information and {len(filtered_results)} records with camera model information.")

There are 497 records with missing camera model information and 2795 records with camera model information.


In [6]:
joined_results = filtered_results.merge(top_records_df, on='catalog_number', how='left')
joined_results.to_csv("../results/camera_metadata.csv", index=False)